Problem formulation and evaluation methods is from [here](https://arxiv.org/pdf/1505.04094.pdf)

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
df_orders = pd.read_csv('data/instacart-market-basket-analysis/orders.csv')
# df_products = pd.read_csv('data/instacart-market-basket-analysis/products.csv')
# df_aisles = pd.read_csv('data/instacart-market-basket-analysis/aisles.csv')
# df_departments = pd.read_csv('data/instacart-market-basket-analysis/departments.csv')
df_order_products_prior = pd.read_csv('data/instacart-market-basket-analysis/order_products__prior.csv')
# df_order_products_train = pd.read_csv('data/instacart-market-basket-analysis/order_products__train.csv')

In [7]:
df_orders = df_orders[df_orders['eval_set'] == 'prior']
df_orders = df_orders.drop(['eval_set'], axis=1)
df_orders.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0


In [19]:
order_counts = df_orders.groupby("user_id").size().sort_values(ascending=False).to_frame('order_count')

In [20]:
order_counts[order_counts["order_count"] >= 10]

,order_count
user_id,
152340,99
185641,99
185524,99
81678,99
70922,99
...,...
201946,10
201820,10
187122,10


In [24]:
df_orders_clean = df_orders.join(order_counts[order_counts["order_count"] >= 10], on="user_id", how="inner")
df_orders_clean

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_count
0,2539329,1,1,2,8,NaN,10
1,2398795,1,2,3,7,15.0,10
2,473747,1,3,3,12,21.0,10
3,2254736,1,4,4,7,29.0,10
4,431534,1,5,4,15,28.0,10
...,...,...,...,...,...,...,...
3421077,2558525,206209,9,4,15,22.0,13
3421078,2266710,206209,10,5,18,29.0,13
3421079,1854736,206209,11,4,10,30.0,13
3421080,626363,206209,12,1,12,18.0,13


In [31]:
df_orders_clean["train_feature"] = (df_orders_clean["order_number"] <= df_orders_clean["order_count"] - 3).astype(int)
df_orders_clean["train_label"] = (df_orders_clean["order_number"] == df_orders_clean["order_count"] - 2).astype(int)

df_orders_clean["valid_feature"] = (df_orders_clean["order_number"] <= df_orders_clean["order_count"] - 2).astype(int)
df_orders_clean["valid_label"] = (df_orders_clean["order_number"] == df_orders_clean["order_count"] - 1).astype(int)

df_orders_clean["test_feature"] = (df_orders_clean["order_number"] <= df_orders_clean["order_count"] - 1).astype(int)
df_orders_clean["test_label"] = (df_orders_clean["order_number"] == df_orders_clean["order_count"]).astype(int)

In [32]:
df_orders_clean[df_orders_clean["user_id"] == 1]

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_count,train_feature,train_label,test_feature,test_label
0,2539329,1,1,2,8,NaN,10,1,0,1,0
1,2398795,1,2,3,7,15.0,10,1,0,1,0
2,473747,1,3,3,12,21.0,10,1,0,1,0
3,2254736,1,4,4,7,29.0,10,1,0,1,0
4,431534,1,5,4,15,28.0,10,1,0,1,0
5,3367565,1,6,2,7,19.0,10,1,0,1,0
6,550135,1,7,1,9,20.0,10,1,0,1,0
7,3108588,1,8,1,14,14.0,10,1,0,1,0
8,2295261,1,9,1,16,0.0,10,0,1,1,0
9,2550362,1,10,4,8,30.0,10,0,0,0,1


In [57]:
train_features = df_orders_clean[df_orders_clean["train_feature"] == 1].drop(
        ["train_feature", "train_label", "test_feature", "test_label"], axis=1
    ).join(
        df_order_products_prior.set_index("order_id"), on="order_id", how="inner"
    )
print("feature df shape: ", train_features.shape)

feature df shape:  (24765325, 10)


In [60]:
train_features

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_count,product_id,add_to_cart_order,reordered
0,2539329,1,1,2,8,NaN,10,196,1,0
0,2539329,1,1,2,8,NaN,10,14084,2,0
0,2539329,1,1,2,8,NaN,10,12427,3,0
0,2539329,1,1,2,8,NaN,10,26088,4,0
0,2539329,1,1,2,8,NaN,10,26405,5,0
...,...,...,...,...,...,...,...,...,...,...
3421079,1854736,206209,11,4,10,30.0,13,24852,4,1
3421079,1854736,206209,11,4,10,30.0,13,19348,5,1
3421079,1854736,206209,11,4,10,30.0,13,15700,6,1
3421079,1854736,206209,11,4,10,30.0,13,40310,7,0


In [58]:
train_edges = train_features.groupby(["user_id", "product_id"]).agg({"order_number": "count", "order_count":"mean"}).reset_index()
train_edges["weight"]  = train_edges["order_number"] / train_edges["order_count"]
train_edges = train_edges.drop(["order_number", "order_count"], axis=1)
print("Edge shape: ", train_edges.shape)
train_edges.head()

Edge shape:  (9006247, 3)


,user_id,product_id,weight
0,1,196,0.8
1,1,10258,0.7
2,1,10326,0.1
3,1,12427,0.8
4,1,13032,0.2


In [50]:
train_labels = df_orders_clean[df_orders_clean["train_label"] == 1].drop(
    ["train_feature", "train_label", "test_feature", "test_label"], axis=1
    ).join(
        df_order_products_prior.set_index("order_id"), on="order_id", how="inner"
    )
train_labels = train_labels.groupby(["user_id", "product_id"]).agg({"order_number": "count"}).reset_index()
train_labels = train_labels.rename(columns={"order_number": "label"})
train_labels

,user_id,product_id,label
0,1,196,1
1,1,10258,1
2,1,12427,1
3,1,25133,1
4,1,46149,1
...,...,...,...
1093107,206209,33351,1
1093108,206209,38167,1
1093109,206209,41213,1
1093110,206209,43961,1


In [51]:
train_labels = pd.merge(train_labels, train_edges, on=["user_id", "product_id"], how="left")
train_labels = train_labels[train_labels["weight"].isnull()]
train_labels = train_labels.drop(["weight"], axis=1).reset_index(drop=True)
train_labels

,user_id,product_id,label
0,2,79,1
1,2,5869,1
2,2,16521,1
3,2,19057,1
4,2,30908,1
...,...,...,...
330834,206209,14727,1
330835,206209,16168,1
330836,206209,20590,1
330837,206209,26209,1


In [56]:
9337086 - 330839

9006247

In [52]:
test_features = df_orders_clean[df_orders_clean["test_feature"] == 1].drop(
        ["train_feature", "train_label", "test_feature", "test_label"], axis=1
    ).join(
        df_order_products_prior.set_index("order_id"), on="order_id", how="inner"
    )
print("feature df shape: ", test_features.shape)

feature df shape:  (25858437, 10)


In [53]:
test_edges = test_features.groupby(["user_id", "product_id"]).agg({"order_number": "count", "order_count":"mean"}).reset_index()
test_edges["weight"]  = test_edges["order_number"] / test_edges["order_count"]
test_edges = test_edges.drop(["order_number", "order_count"], axis=1)
print("Edge shape: ", test_edges.shape)
test_edges.head()

Edge shape:  (9337086, 3)


,user_id,product_id,weight
0,1,196,0.9
1,1,10258,0.8
2,1,10326,0.1
3,1,12427,0.9
4,1,13032,0.2


In [54]:
test_labels = df_orders_clean[df_orders_clean["test_label"] == 1].drop(
    ["train_feature", "train_label", "test_feature", "test_label"], axis=1
    ).join(
        df_order_products_prior.set_index("order_id"), on="order_id", how="inner"
    )
test_labels = test_labels.groupby(["user_id", "product_id"]).agg({"order_number": "count"}).reset_index()
test_labels = test_labels.rename(columns={"order_number": "label"})
test_labels

,user_id,product_id,label
0,1,196,1
1,1,10258,1
2,1,12427,1
3,1,13032,1
4,1,25133,1
...,...,...,...
1100488,206209,22920,1
1100489,206209,24852,1
1100490,206209,31477,1
1100491,206209,38730,1


In [55]:
test_labels = pd.merge(test_labels, test_edges, on=["user_id", "product_id"], how="left")
test_labels = test_labels[test_labels["weight"].isnull()]
test_labels = test_labels.drop(["weight"], axis=1).reset_index(drop=True)
test_labels

,user_id,product_id,label
0,1,35951,1
1,1,38928,1
2,1,39657,1
3,2,7963,1
4,2,20785,1
...,...,...,...
328198,206209,6567,1
328199,206209,22920,1
328200,206209,31477,1
328201,206209,38730,1


In [59]:
train_features.to_parquet("data/train_features.parquet")
train_labels.to_parquet("data/train_labels.parquet")
train_edges.to_parquet("data/train_edges.parquet")
test_features.to_parquet("data/test_features.parquet")
test_labels.to_parquet("data/test_labels.parquet")
test_edges.to_parquet("data/test_edges.parquet")